In [2]:
!pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd drive/MyDrive/data

/content/drive/MyDrive/data


In [62]:
!pip install pandas --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
import pandas as pd

train_df = pd.read_csv("train.csv")
train_df.head()

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise


In [5]:
train_df.groupby(by=['Target']).count()

,ID,Utterance,Speaker,Dialogue_ID
Target,,,,
anger,1109,1109,1109,1109
disgust,271,271,271,271
fear,268,268,268,268
joy,1743,1743,1743,1743
neutral,4710,4710,4710,4710
sadness,683,683,683,683
surprise,1205,1205,1205,1205


In [66]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
label_encoder.fit(train_df['Target'])
num_labels = len(label_encoder.classes_)

train_df['encoded_target'] = np.asarray(label_encoder.transform(train_df['Target']), dtype=np.int32)

In [67]:
train_df.head()

,ID,Utterance,Speaker,Dialogue_ID,Target,encoded_target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral,4
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral,4
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral,4
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral,4
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise,6


In [68]:
train_texts = train_df['Utterance'].to_list()
train_labels = train_df['encoded_target'].to_list()

In [69]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, 
                                                                    test_size=0.2, random_state=42)

In [70]:
train_tokenizer = tokenizer(train_texts, truncation = True, padding = True, max_length = 128)
val_tokenizer = tokenizer(val_texts, truncation = True, padding = True, max_length = 128)

### define model

In [1]:
!pip install transformers -q

     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 84.4 MB/s 
     |████████████████████████████████| 7.6 MB 75.6 MB/s 


In [10]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

model_name = "tae898/emoberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case = True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels)

In [11]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

In [15]:
from transformers import TFAutoModel
from transformers import TFAutoModelForSequenceClassification

In [13]:
tokenizer = AutoTokenizer.from_pretrained(pt_save_directory)

In [16]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained(pt_save_directory, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [71]:
tf_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 354310144 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 1056775   
 ficationHead)                                                   
                                                                 
Total params: 355,366,919
Trainable params: 355,366,919
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:92: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


### go

In [ ]:
!pip install tensorflow_addons

In [72]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_tokenizer), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_tokenizer), val_labels))

In [82]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
tf_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [79]:
print(train_dataset)
print(val_dataset)

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(62,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(62,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>
<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(92,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(92,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [83]:
tf_model.fit(
    train_dataset.shuffle(1000).batch(16), 
    epochs=5, 
    batch_size=16, 
    validation_data=val_dataset.shuffle(1000).batch(16)
)

Epoch 1/5
500/500 [==============================] - 107s 147ms/step - loss: 2.6866 - accuracy: 0.2935 - val_loss: 2.3208 - val_accuracy: 0.4930
Epoch 2/5
500/500 [==============================] - 67s 133ms/step - loss: 2.3026 - accuracy: 0.2525 - val_loss: 2.1369 - val_accuracy: 0.0275
Epoch 3/5
500/500 [==============================] - 69s 137ms/step - loss: 2.2358 - accuracy: 0.1254 - val_loss: 2.1355 - val_accuracy: 0.0275
Epoch 4/5
500/500 [==============================] - 66s 133ms/step - loss: 2.2337 - accuracy: 0.2142 - val_loss: 2.1328 - val_accuracy: 0.0275
Epoch 5/5
500/500 [==============================] - 67s 135ms/step - loss: 1.9491 - accuracy: 0.2375 - val_loss: 1.9403 - val_accuracy: 0.4930


In [84]:
test_df = pd.read_csv("test.csv")
test_df.head()

,ID,Utterance,Speaker,Dialogue_ID
0,TEST_0000,Why do all the coffee cups have figures below?,Mark,0
1,TEST_0001,"Oh. It's so Monica can follow. Of this way, if...",Rachell,0
2,TEST_0002,You know what?,Rachell,0
3,TEST_0003,"Come on, Lydia, you can do it.",Joeyy,1
4,TEST_0004,To push!,Joeyy,1


In [ ]:
from transformers import TextClassificationPipeline

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=tf_model, 
    framework='tf',
    return_all_scores=True
)

In [87]:
preds = []

for text in test_df['Utterance']:
    preds.append(text_classifier(text)[0])


KeyboardInterrupt: ignored

# 씨발

In [88]:
from transformers import TFTrainer, TFTrainingArguments

In [90]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs'            # directory for storing logs
)

with training_args.strategy.scope():
    trainer_model = tf_model

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:120: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  FutureWarning,


In [91]:
trainer.train()

ValueError: ignored